In [80]:
from martian_apart_hack_sdk import exceptions, judge_specs, martian_client, utils
from martian_apart_hack_sdk.models import judge_evaluation, llm_models, router_constraints
import pickle
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple
import uuid
import time

from openai.types.chat import (
    chat_completion,
    chat_completion_message,
)

In [30]:
config = utils.load_config()
client = martian_client.MartianClient(
    api_url=config.api_url,
    api_key=config.api_key,
)

In [31]:
JUDGE_IDS = [
    'harmlessness-judge',
    'privacy-judge',
    'factual-accuracy-judge',
    'prompt-faithfulness-relevance-judge',
    'calibration-uncertainty-judge',
    'bias-fairness-judge',
    'reasoning-consistency-judge',
    'discourse-coherence-judge',
    'conciseness-redundancy-judge',
    'style-formatting-judge',
]

In [32]:
JUDGES = { id: client.judges.get(judge_id=id) for id in JUDGE_IDS }

In [33]:
def get_score(query, answer, judge):
    # Run the judge spec.
    chat_request_text = query
    chat_response_text = answer

    completion_request = {
        "model": llm_models.GPT_4O_MINI,
        "messages": [{"role": "user", "content": chat_request_text}],
    }

    chat_completion_response = chat_completion.ChatCompletion(
        id="123",
        choices=[
            chat_completion.Choice(
                finish_reason="stop",
                index=0,
                message=chat_completion_message.ChatCompletionMessage(
                    role="assistant",
                    content=chat_response_text,
                ),
            )
        ],
        created=0,
        model="gpt-4o",
        object="chat.completion",
        service_tier=None,
    )

    evaluation_result = client.judges.evaluate(
        judge,
        completion_request=completion_request,
        completion_response=chat_completion_response,
    )

    return evaluation_result.score

## Load data

In [76]:
with open('data_with_human_feedback.pkl', 'rb') as f:
    reconstructed = pickle.load(f)

In [2]:
import pickle

In [3]:
with open('data_with_judge_scores.pkl', 'rb') as f:
    miracle = pickle.load(f)

In [4]:
miracle

,source,instruction,model,answer,score,grader,human_feedback,human_feedback_score,human_feedback_analysis,persona_name,persona_bio,scores
uuid,,,,,,,,,,,,
dd107a11-0cef-469f-861d-1f6c378f616a,flan_v2_niv2,The task is about translation from Telugu to E...,wizardlm-70b,[A]: Oh no!,6.0,wizardlm-70b,"{'analysis': 'The answer conveys a sudden, neg...",7,"The answer conveys a sudden, negative reaction...",Luna Martinez - Novelist,"Luna Martinez, a novelist and poet who appreci...","[3.5, 4, 0.9, 0.9, 0.8, 3, 1.5, 1.5, 3.5, 2.5]"
a8585e02-6395-4064-b053-6a5e6bf903c6,sharegpt,can you explain this alert configuration?\n\nC...,llama-2-70b-chat,"Sure, I'd be happy to explain this alert confi...",7.0,llama-2-70b-chat,{'analysis': 'The explanation is thorough but ...,6,"The explanation is thorough but dry, lacking a...",Luna Martinez - Novelist,"Luna Martinez, a novelist and poet who appreci...","[3.8, 4, 3.5, 3.5, 2.5, 4, 3.8, 3.8, 2.8, 3.2]"
0eeb0570-8ba3-4844-b205-b1f1318cabf6,flan_v2_niv2,Q: You are given a sentence in Arabic. Your jo...,vicuna-33b,"Per questo, non esiste una strategia che si co...",9.0,vicuna-33b,{'analysis': 'The translation is clear and con...,7,"The translation is clear and concise, conveyin...",Luna Martinez - Novelist,"Luna Martinez, a novelist and poet who appreci...","[4, 3.8, 4, 3.5, 1, 3.5, 3.8, 3.5, 2.5, 3.5]"
4fb95660-d3d2-482a-ab81-33c3f4171e26,evol_instruct,Can you suggest some specific actions or strat...,llama-2-70b-chat,Thank you for asking! I'm happy to help you wi...,8.5,llama-2-70b-chat,{'analysis': 'The answer provides a comprehens...,6,The answer provides a comprehensive list of st...,Sarah Chen - University Professor,"Dr. Sarah Chen, a rigorous university professo...","[4, 4, 3, 3.5, 2.5, 3.5, 3.5, 3.5, 2.5, 3.5]"
f20121e8-f572-46bf-95b7-f8a988c6629f,sharegpt,I want you to act as a Content writer very pro...,wizardlm-7b,Hammocks and swings can be a great choice for ...,6.0,wizardlm-7b,{'analysis': 'The article provides a clear int...,6,The article provides a clear introduction to h...,Jamie Park - College Student,"Jamie Park, an enthusiastic college junior who...","[3.5, 3.8, 1.5, 1.5, 2.5, 2.5, 2, 2.5, 2.5, 0.5]"
...,...,...,...,...,...,...,...,...,...,...,...,...
53ab6265-0d1c-4711-863b-884022cd5444,flan_v2_niv2,You will be given a definition of a task first...,llama-2-70b-chat,Keyword: Intestinal Ischemia\n\nConfidence: 90...,8.5,llama-2-70b-chat,{'analysis': 'The answer directly addresses th...,9,The answer directly addresses the main point o...,Elena Rodriguez - Working Parent,"Elena Rodriguez, a working mother of three who...","[3.8, 4, 2.5, 3.5, 2.7, 3.8, 3.5, 3.5, 2.5, 3.5]"
aee56f97-1bbc-4882-a356-e31256c2a853,flan_v2_cot,q: James was driving away from work. He slid i...,wizardlm-13b,fountain\nThe answer is fountain\n\nQUESTION: ...,7.0,wizardlm-13b,{'analysis': 'The candidate answers demonstrat...,6,The candidate answers demonstrate clear and lo...,Amara Johnson - Therapist,"Dr. Amara Johnson, a therapist who prioritizes...","[3.5, 4, 0.8, 2.5, 1.5, 3.5, 1.5, 2.5, 2.5, 1.5]"
472aeffb-64c8-4ea0-82f7-ccbdc6787fe9,sharegpt,Write the complete script of a Seinfeld episod...,wizardlm-13b,[Jerry's Apartment]\n\n[Jerry and George are s...,6.0,wizardlm-13b,{'analysis': 'The script effectively uses humo...,7,The script effectively uses humor to explain a...,Amara Johnson - Therapist,"Dr. Amara Johnson, a therapist who prioritizes...","[3.8, 4, 2.5, 3.5, 1.5, 3.5, 3.5, 3.5, 1.5, 3]"


In [36]:
def evaluate_single_judge(args: Tuple[str, str, str, dict]) -> Tuple[int, float]:
    """
    Evaluate a single judge - designed for parallel execution
    
    Args:
        args: Tuple of (question, answer, judge_id, judges_dict)
    Returns:
        Tuple of (judge_index, score)
    """
    question, answer, judge_id, judges = args
    judge_index = JUDGE_IDS.index(judge_id)
    score = get_score(question, answer, judges[judge_id])
    return (judge_index, score)

In [84]:
def evaluate_with_backoff(args, max_retries: int = 5, initial_delay: float = 1.0):
    """
    Wrap evaluate_single_judge with exponential back-off on exception.
    
    Args:
        args: Tuple of (question, answer, judge_id, JUDGES)
        max_retries: Number of retry attempts
        initial_delay: Seconds to sleep before first retry
    """
    delay = initial_delay
    for attempt in range(max_retries):
        try:
            return evaluate_single_judge(args)
        except Exception:
            if attempt == max_retries - 1:
                # Last attempt: re-raise or return a default
                raise
            time.sleep(delay)
            delay *= 2
    # Fallback (should not reach here)
    return evaluate_single_judge(args)

In [85]:
def evaluate_pair_parallel(question: str, answer: str, max_workers: int = None) -> List[float]:
    """
    Parallelize judge evaluations using ThreadPoolExecutor,
    with exponential back-off on each judge call.
    
    Args:
        question: The query text
        answer: The response text
        max_workers: Number of parallel workers (None = auto-select based on CPU)
    
    Returns:
        List of scores in judge order
    """
    scores = [0.0] * len(JUDGE_IDS)
    eval_args = [(question, answer, judge_id, JUDGES) for judge_id in JUDGE_IDS]
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for judge_index, score in executor.map(evaluate_with_backoff, eval_args):
            scores[judge_index] = score
    
    return scores

In [89]:
def eval_row(args):
    idx, question, answer = args
    return idx, evaluate_pair_parallel(question, answer)

# Prepare argument list
results = [None] * len(reconstructed)
tasks = [
    (i, row["instruction"], row["answer"])
    for i, (_, row) in enumerate(reconstructed.iterrows())
]

# Parallelize across rows
with ThreadPoolExecutor() as executor:
    for i, scores in executor.map(eval_row, tasks):
        results[i] = scores

reconstructed['scores'] = results

In [91]:
with open('data_with_judge_scores.pkl', 'wb') as f:
    pickle.dump(reconstructed, f)

In [90]:
reconstructed

,source,instruction,model,answer,score,grader,human_feedback,human_feedback_score,human_feedback_analysis,persona_name,persona_bio,scores
uuid,,,,,,,,,,,,
dd107a11-0cef-469f-861d-1f6c378f616a,flan_v2_niv2,The task is about translation from Telugu to E...,wizardlm-70b,[A]: Oh no!,6.0,wizardlm-70b,"{'analysis': 'The answer conveys a sudden, neg...",7,"The answer conveys a sudden, negative reaction...",Luna Martinez - Novelist,"Luna Martinez, a novelist and poet who appreci...","[3.5, 4, 0.9, 0.9, 0.8, 3, 1.5, 1.5, 3.5, 2.5]"
a8585e02-6395-4064-b053-6a5e6bf903c6,sharegpt,can you explain this alert configuration?\n\nC...,llama-2-70b-chat,"Sure, I'd be happy to explain this alert confi...",7.0,llama-2-70b-chat,{'analysis': 'The explanation is thorough but ...,6,"The explanation is thorough but dry, lacking a...",Luna Martinez - Novelist,"Luna Martinez, a novelist and poet who appreci...","[3.8, 4, 3.5, 3.5, 2.5, 4, 3.8, 3.8, 2.8, 3.2]"
0eeb0570-8ba3-4844-b205-b1f1318cabf6,flan_v2_niv2,Q: You are given a sentence in Arabic. Your jo...,vicuna-33b,"Per questo, non esiste una strategia che si co...",9.0,vicuna-33b,{'analysis': 'The translation is clear and con...,7,"The translation is clear and concise, conveyin...",Luna Martinez - Novelist,"Luna Martinez, a novelist and poet who appreci...","[4, 3.8, 4, 3.5, 1, 3.5, 3.8, 3.5, 2.5, 3.5]"
4fb95660-d3d2-482a-ab81-33c3f4171e26,evol_instruct,Can you suggest some specific actions or strat...,llama-2-70b-chat,Thank you for asking! I'm happy to help you wi...,8.5,llama-2-70b-chat,{'analysis': 'The answer provides a comprehens...,6,The answer provides a comprehensive list of st...,Sarah Chen - University Professor,"Dr. Sarah Chen, a rigorous university professo...","[4, 4, 3, 3.5, 2.5, 3.5, 3.5, 3.5, 2.5, 3.5]"
f20121e8-f572-46bf-95b7-f8a988c6629f,sharegpt,I want you to act as a Content writer very pro...,wizardlm-7b,Hammocks and swings can be a great choice for ...,6.0,wizardlm-7b,{'analysis': 'The article provides a clear int...,6,The article provides a clear introduction to h...,Jamie Park - College Student,"Jamie Park, an enthusiastic college junior who...","[3.5, 3.8, 1.5, 1.5, 2.5, 2.5, 2, 2.5, 2.5, 0.5]"
...,...,...,...,...,...,...,...,...,...,...,...,...
53ab6265-0d1c-4711-863b-884022cd5444,flan_v2_niv2,You will be given a definition of a task first...,llama-2-70b-chat,Keyword: Intestinal Ischemia\n\nConfidence: 90...,8.5,llama-2-70b-chat,{'analysis': 'The answer directly addresses th...,9,The answer directly addresses the main point o...,Elena Rodriguez - Working Parent,"Elena Rodriguez, a working mother of three who...","[3.8, 4, 2.5, 3.5, 2.7, 3.8, 3.5, 3.5, 2.5, 3.5]"
aee56f97-1bbc-4882-a356-e31256c2a853,flan_v2_cot,q: James was driving away from work. He slid i...,wizardlm-13b,fountain\nThe answer is fountain\n\nQUESTION: ...,7.0,wizardlm-13b,{'analysis': 'The candidate answers demonstrat...,6,The candidate answers demonstrate clear and lo...,Amara Johnson - Therapist,"Dr. Amara Johnson, a therapist who prioritizes...","[3.5, 4, 0.8, 2.5, 1.5, 3.5, 1.5, 2.5, 2.5, 1.5]"
472aeffb-64c8-4ea0-82f7-ccbdc6787fe9,sharegpt,Write the complete script of a Seinfeld episod...,wizardlm-13b,[Jerry's Apartment]\n\n[Jerry and George are s...,6.0,wizardlm-13b,{'analysis': 'The script effectively uses humo...,7,The script effectively uses humor to explain a...,Amara Johnson - Therapist,"Dr. Amara Johnson, a therapist who prioritizes...","[3.8, 4, 2.5, 3.5, 1.5, 3.5, 3.5, 3.5, 1.5, 3]"
